In [9]:
import pandas as pd
import numpy as np
import re
from dataLoader import *
from utils import *
import argparse
import yaml
import os

In [10]:
def category_correspondence(selected_terms, categories, ename2eid, eid2DocProb, eidDocPair2Prob):
    score = 0
    for cat in categories:
        if cat not in ename2eid: continue
        scores = []
        for term in selected_terms:
            scores += [get_nmi(ename2eid[cat], ename2eid[term], eid2DocProb, eidDocPair2Prob)]
        score += sum(scores)
    return score

In [11]:
def result(config_file):
    with open(config_file, 'r') as ymlfile:
        config = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
    domain_path = config['dataset']['domain_path']
    result_path = os.path.join(config['dataset']['domain_path'], config['dataset']['result_folder'])
    
    eid2ename, ename2eid = loadEidToEntityMap(domain_path + 'intermediate/entity2id.txt')
    eid2DocProb = loadEid2DocFeature(domain_path + 'intermediate/eid2DocProb.txt')
    eidDocPair2Prob = loadEidDocPairFeature(domain_path + 'intermediate/eidDocPair2prob.txt')
    
    gt_file = '../data/groundtruths/category_correspondence/arxivcs_categories.txt'
    cs_cates = []
    with open(gt_file, 'r') as f:
        for line in f:
            cat = line.strip()
            cat = cat.lower()
            cat = re.sub(r'[^\x00-\x7F]+', ' ', cat)
            cat = cat.replace("-", " ")
            cat = "_".join(cat.split())
            cs_cates.append(cat)
    cs_cates = np.array(cs_cates)
    
    selected_terms_rf = pd.read_csv(result_path+'rf.txt', header=None, sep='\n').values[:,0]
    selected_terms_lo = pd.read_csv(result_path+'lo.txt', header=None, sep='\n').values[:,0]
    selected_terms_fl = pd.read_csv(result_path+'fl.txt', header=None, sep='\n').values[:,0]
    selected_terms_pr = pd.read_csv(result_path+'pr.txt', header=None, sep='\n').values[:,0]
    selected_terms_kl_rf = pd.read_csv(result_path+'kl_rf.txt', header=None, sep='\n').values[:,0]
    selected_terms_mm = pd.read_csv(result_path+'mm.txt', header=None, sep='\n').values[:,0]
    selected_terms_kl_mm = pd.read_csv(result_path+'kl_mm.txt', header=None, sep='\n').values[:,0]
    
    
    ks = [10, 20, 30, 40, 50, 100, 200, 500]
    ccs = []
    for k in ks:
        cc = []
        cc.append(category_correspondence(selected_terms_rf[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        cc.append(category_correspondence(selected_terms_lo[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        cc.append(category_correspondence(selected_terms_fl[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        cc.append(category_correspondence(selected_terms_kl_rf[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        cc.append(category_correspondence(selected_terms_mm[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        cc.append(category_correspondence(selected_terms_kl_mm[:k], cs_cates, ename2eid, eid2DocProb, eidDocPair2Prob))
        ccs.append(", ".join([str(round(c,4)) for c in cc]))
        
        print(ccs[-1])

In [12]:
config_file = "configs/arxivcs_ap.yaml" # candidate keywords: authoprhase extracted keywords
result(config_file)

Loading: ../data/arxiv/cs/all_ap/intermediate/entity2id.txt: 100%|██████████| 93148/93148 [00:00<00:00, 716882.69it/s]
Loading: ../data/arxiv/cs/all_ap/intermediate/eid2DocProb.txt: 100%|██████████| 93148/93148 [00:00<00:00, 705389.08it/s]
Loading: ../data/arxiv/cs/all_ap/intermediate/eidDocPair2prob.txt: 100%|██████████| 13653903/13653903 [00:51<00:00, 264724.16it/s]


1.0651, 1.1001, 1.0722, 1.0651, 2.0981, 2.1212
1.144, 3.2476, 1.1345, 1.1451, 4.2626, 4.2972
2.2134, 4.3965, 3.2682, 3.2875, 4.4321, 4.4169
3.3273, 4.4929, 3.3515, 3.366, 4.6, 4.6018
3.453, 4.6896, 3.4505, 3.4496, 5.6826, 5.6826
4.7812, 8.2382, 4.7626, 4.8761, 8.2709, 8.2858
9.7166, 11.105, 9.5908, 8.7045, 11.109, 12.0209
18.8958, 19.3732, 16.6155, 17.7429, 19.3248, 19.2368


In [13]:
config_file = "configs/arxivcs_sp.yaml" # candidate keywords: springer
result(config_file)

Loading: ../data/arxiv/cs/all_sp/intermediate/entity2id.txt: 100%|██████████| 64160/64160 [00:00<00:00, 802876.52it/s]
Loading: ../data/arxiv/cs/all_sp/intermediate/eid2DocProb.txt: 100%|██████████| 64160/64160 [00:00<00:00, 750273.35it/s]
Loading: ../data/arxiv/cs/all_sp/intermediate/eidDocPair2prob.txt: 100%|██████████| 16978645/16978645 [00:54<00:00, 312408.07it/s]


2.0691, 2.1157, 1.0852, 1.0997, 2.1402, 2.1432
2.1745, 2.2393, 2.1745, 2.1805, 2.2916, 3.3261
2.2302, 3.3939, 2.2261, 2.2556, 3.4261, 3.4265
2.2873, 3.5047, 2.2848, 2.3224, 5.5382, 5.5406
2.3592, 5.6138, 2.3592, 2.369, 5.6348, 5.6578
3.7702, 7.1071, 3.7603, 3.7848, 8.1346, 7.1284
6.4212, 12.031, 5.3436, 6.3999, 12.1114, 12.0714
13.3435, 19.4273, 13.2773, 12.25, 19.5075, 19.4891


In [14]:
config_file = "configs/arxivcs_am.yaml" # candidate keywords: aminer
result(config_file)

Loading: ../data/arxiv/cs/all_am/intermediate/entity2id.txt: 100%|██████████| 44938/44938 [00:00<00:00, 756479.33it/s]
Loading: ../data/arxiv/cs/all_am/intermediate/eid2DocProb.txt: 100%|██████████| 44938/44938 [00:00<00:00, 764971.54it/s]
Loading: ../data/arxiv/cs/all_am/intermediate/eidDocPair2prob.txt: 100%|██████████| 14735109/14735109 [00:52<00:00, 281828.92it/s]


1.102, 2.1205, 1.1073, 1.1109, 2.1205, 2.1645
2.1782, 2.305, 2.1699, 2.1956, 3.3048, 3.3079
2.2433, 3.432, 2.2508, 2.2795, 3.4394, 4.4522
2.3159, 3.5312, 2.3012, 2.3944, 5.5693, 5.57
2.4617, 5.6519, 3.4657, 2.4675, 5.6463, 5.6829
3.8432, 8.1523, 3.8038, 3.8429, 9.1788, 8.1679
6.5778, 12.1533, 6.532, 6.5431, 12.2007, 12.1013
13.6675, 19.5232, 13.4622, 12.4942, 19.558, 20.5255
